In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import sklearn
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42
LABELS = ["Normal", "Fraud"]

In [3]:

data = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv',sep=',')
data.head()

In [4]:
data.info()

In [5]:
## Take some sample of the data

data1= data.sample(frac = 0.1,random_state=1)

data1.shape

In [6]:
#Determine the number of fraud and valid transactions in the dataset

Fraud = data1[data1['Class']==1]

Valid = data1[data1['Class']==0]

outlier_fraction = len(Fraud)/float(len(Valid))

In [7]:

print(outlier_fraction)

print("Fraud Cases : {}".format(len(Fraud)))

print("Valid Cases : {}".format(len(Valid)))

In [8]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
data1.head()

In [9]:
method = SMOTE(k_neighbors = 2)
X = data1.drop(['Class'],axis=1)
y = data1['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y,  train_size=0.8, random_state=0) 

#Apply resampling to the training data only
#X_resampled, y_resampled = method.fit_sample(X_train, y_train) 


Random forest bieng applied

In [10]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score
model = RandomForestClassifier(random_state=42) 
model.fit(X_train, y_train) 
predicted = model.predict(X_test) 
print (accuracy_score(y_test, predicted)) 


In [11]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression(random_state=0,max_iter=100,solver='liblinear')
model1.fit(X_train, y_train) 
predicted1 = model.predict(X_test)

 Fraud Detection in Python Obtaining performance metrics

In [12]:
# Import the packages 
from sklearn.metrics import precision_recall_curve 
from sklearn.metrics import average_precision_score 
# Calculate average precision and the PR curve 
average_precision = average_precision_score(y_test, predicted) 
# Obtain precision and recall 
precision, recall, _ = precision_recall_curve(y_test, predicted)
# Print classification report using predictions 

print(classification_report(y_test, predicted)) 

# You Can Also Predict Probabilities from model


In [13]:
#probabilities = model.predict_proba(Xtest)

In [14]:
from sklearn.metrics import confusion_matrix
# Print confusion matrix using predictions 

print(confusion_matrix(y_test, predicted)) 

Here we use ensemble model

In [15]:
from sklearn.ensemble import VotingClassifier

clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)

emsemble_model = VotingClassifier(estimators=[('lr',clf1),('rf',clf2)])


In [16]:
emsemble_model.fit(X_train, y_train)
emsemble_model.predict(X_test)


In [17]:
#printing the confusion matrix
LABELS = ['Normal','Fraud']
conf_matrix = confusion_matrix(y_test, predicted)
plt.figure(figsize=(12, 12))
sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt='d')
plt.title('Confusion matrix')
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

Hyperparameter Tuning

In [18]:
from sklearn.model_selection import GridSearchCV 
# Create the parameter grid
param_grid = {     
    'max_depth': [2, 3, 4],     
    'max_features': [2, 3],     
    'min_samples_leaf': [3, 4, 5],     
    'min_samples_split': [8, 10, 12],     
    'n_estimators': [2, 3, 4] } 
# Define which model to use 
model = RandomForestClassifier () 
# Instantiate the grid search model
grid_search_model = GridSearchCV(estimator = model,  param_grid = param_grid, cv = 5, n_jobs = -1, scoring='f1') 

DataCamp Fraud Detection in Python Finding the best model with GridSearchCV

In [19]:
grid_search_model.fit(X_train, y_train)

In [20]:
grid_search_model.best_params_

In [21]:
# Get the best_estimator results
grid_search_model.best_estimator_ 
#grid_search_model.best_score_

Visualizing the confusion matrix(Random Forest)

In [22]:
#printing the confusion matrix
LABELS = ['Normal','Fraud']
conf_matrix = confusion_matrix(y_test, predicted)
plt.figure(figsize=(12, 12))
sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt='d')
plt.title('Confusion matrix')
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

Visualizing the confusion matrix(LogisticRegression)

In [23]:
#printing the confusion matrix
LABELS = ['Normal','Fraud']
conf_matrix = confusion_matrix(y_test, predicted1)
plt.figure(figsize=(12, 12))
sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt='d')
plt.title('Confusion matrix')
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

In [24]:
#visualizing RF
model = RandomForestClassifier(max_depth=4, max_features=3, min_samples_leaf=3,
                       min_samples_split=12, n_estimators=6)

# Train
model.fit(X_train, y_train)
# Extract single tree
estimator = model.estimators_[5]

from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = X.columns.tolist(),
                class_names = ['0',' 1'],
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

In [25]:
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
        color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()